# Homework 5

In [1]:
#include <iostream>
#include <vector>
using namespace std;

## template
### `swap()`
先看一个简单的交换两个`int`型变量的函数：

In [2]:
void swap(int& a, int& b)
{
    int temp = a;
    a =  b;
    b = temp;
}

In [3]:
int ia = 1;
int ib = 2;

swap(ia, ib);
cout << "ia: " << ia << ", ib: " << ib;

ia: 2, ib: 1

可是上面的`swap()`只能接受`int`型参数，对于其他类型的参数还需要定义新的`swap()`，如下面的`swap_float`：

In [4]:
float fa = 1.1;
float fb = 1.2;

void swap_float(float &a, float &b);

### `template_swap()`
为了每一种类型都实现一个对应的`swap()`非常麻烦，如果以后用户自定义了新类型还得修改原代码，`template`可以解决这个问题：

In [5]:
template<typename T> 
void t_swap(T& t1, T& t2)
{
    T tmpT;
    tmpT = t1;
    t1 = t2;
    t2 = tmpT;
}

使用`template`和普通函数的区别在于，函数的参数没有被指定，而是用了一个`T`代替，在调用函数时编译器会判断传入参数的类型：

In [6]:
t_swap(ia, ib);
cout << "ia: " << ia << ", ib: " << ib;

ia: 1, ib: 2

`t_swap`成功交换了`int`型变量，再试一下`float`类型：

In [7]:
t_swap<float> (fa, fb);
cout << "fa: " << fa << ", fb: " << fb;

fa: 1.2, fb: 1.1

### template陷阱
`template`很强大，但是必然也有需要注意的地方，这里我们用`template`实现一个可以计算任意类型的数组的平均值：

In [8]:
template <typename T>
T avg(const T *beg, const T *end)
{
    T sum{};
    int cnt = 0;
    while (beg != end)
    {
        sum += *beg;
        ++beg;
        ++cnt;
    }
    return sum / cnt;
}

**【重点】**思考：
- `T sum{}`中的`{}`作用？
- 能否用`T sum = 0`代替？

In [9]:
int a[] = {1, 2, 3, 4, 5};

cout << "avg: " << avg(a, a+5) << endl;

avg: 3


`char`型字符也可以被计算（ascii码）：
> 提示：`%x`输出16进制

In [10]:
char z = ('1' + '2' + '3') / 3;

printf("%d, %d, %d\n", '1', '2', '3');
printf("%x, %x, %x\n", '1', '2', '3');
printf("%d, %c\n", z, z)

49, 50, 51
31, 32, 33
50, 2


6

如果传入`avg()`是`char`数组会发生什么？
> 提示：`avg()`中的`T sum`会变成`char sum`

In [11]:
// const char *c = "12345";
char c[] = {'1', '2', '3'};
printf("%d, %x\n", avg(c, c+3), avg(c, c+3));

-35, ffffffdd


为什么结果变成了`-35`？

In [12]:
printf("%x\n", -35);
printf("%x\n", (char)(49+50+51)/3);

ffffffdd
ffffffdd


`char`型的范围为`[-128, 127]`：

In [13]:
printf("%x, %x\n", (char)127, (char)128);
printf("%d, %d\n", (char)127, (char)128);

7f, ffffff80
127, -128


## class template
上一节讲了普通函数使用`template`申明参数或返回值类型，下面看下类成员如何使用`template`：

In [14]:
template <class T>
class mypair{
    T a, b;
public:
    mypair(T first, T second){
        a=first;
        b=second;
    }
    T getmax ();
};

在类里使用`template`定义成员变量类型、成员函数参数类型、成员函数返回值类型很简单，下面看下在类外实现成员函数的用法：

In [15]:
template <class T>
T mypair<T>::getmax (){
    T retval;
    retval = a>b ? a : b;
    return retval;
}

In [16]:
// mypair myobject(100, 75);
mypair<int> myobject(100, 75);
cout << myobject.getmax();

100

## 其他
### 专用方法
之前我们定义了`t_swap`函数，可以接受`vector`类型的参数：
> 这里因为jupyter编译器限制，如果实例化了`void t_swap(std::vector<int>& t1, std::vector<int>& t2)`，下面就不能再重新定义了*

In [17]:
vector<int> iv1, iv2;
// t_swap(iv1, iv2);

假设`swap`函数要处理很多元素的`vector<int>`，执行`tmpT = t1`要拷贝`t1`的全部元素，占用大量内存，造成性能下降，可以通过`vector.swap`函数解决这个问题：

In [18]:
template<> 
void t_swap(std::vector<int>& t1, std::vector<int>& t2){
    t1.swap(t2);
    cout << "using vector<int>.swap" << endl;
}

测试一下`std::vector<int>`参数是否匹配成功：

In [19]:
t_swap(iv1, iv2);

using vector<int>.swap


测试一下`std::vector<float>`型参数是否匹配：

In [20]:
vector<float> fv1, fv2;
t_swap(fv1, fv2);

In [21]:
template<class V>
void t_swap(std::vector<V>& t1, std::vector<V>& t2){
    t1.swap(t2);
    cout << "using vector.swap" << endl;
}

In [22]:
t_swap(iv1, iv2);
t_swap(fv1, fv2);

using vector.swap
using vector.swap


### 多个template

In [23]:
template <typename T1, typename T2>
class HetPair{
public:
    //HetPair(){};
    HetPair(T1 t1, T2 t2) : t1_(t1), t2_(t2){};

    void print(){
        cout << "t1_: " << t1_ << "; t2_: " << t2_ << endl;
    };

private:
    T1 t1_;
    T2 t2_;
};

In [24]:
HetPair<int, int> h(1, 2);
h.print();

t1_: 1; t2_: 2
